# <u> Memory-based Collaborative Filtering</u>

This notebook explores **memory-based collaborative filtering** as a first baseline for building a movie recommendation system. The approach relies solely on the raw user–item rating matrix, without the need for training or any machine learning framework.

The general idea is to build recommendations based on similarities, which can be computed in two main ways:

- **User–User Collaborative Filtering**: recommends movies liked by users who have similar preferences and rating behavior to us.
- **Item–Item Collaborative Filtering**: recommends new movies that are similar to the ones we have already seen and liked.

<br>

To define similarities between users or items, we commonly use two metrics:

- **Cosine Similarity**: a fast and popular measure that computes the angle between two vectors (single users or items). The formula is:

<br>

$$
\text{sim}_{\text{cosine}}(x, y) = \frac{x \cdot y}{\|x\| \|y\|}
$$

<br>

- **Pearson Correlation**: more computationally demanding, but it measures the linear relationship between co-rated items, correcting for each user’s individual rating bias. The formula is:

<br>

$$
\text{sim}_{\text{pearson}}(x, y) = \frac{\sum_{i}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum(x_i - \bar{x})^2} \sqrt{\sum(y_i - \bar{y})^2}}
$$

<br>

Due to its ability to correct for individual rating biases, Pearson correlation is especially useful in user–user collaborative filtering, where users may have very different rating scales. In item–item collaborative filtering, this adjustment is less critical, and both Pearson and cosine similarity often produce similar results, particularly in sparse datasets.

## <u>0. Setting:</u>

### <u>0.1 Import libraries and dataframe</u>

In [19]:
# Import necessary libraries
import pandas as pd, numpy as np, os, sys, seaborn as sns, matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

# Set the working directory
current_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(current_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import module for data processing
from modules.data_analysis import *


In [2]:
# Import cleand dataframe
file_path = '../data/processed/ratings_enriched.parquet'
rating_enriched = pd.read_parquet(file_path, engine="pyarrow")
rating_enriched.head(3)

,userId,movieId,rating,timestamp,movie_bayes_avg,log_count_review,release_year,user_avg_rating,user_avg_bayes,title,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2,3.5,2005-04-02 23:53:47,3.212004,10.009828,1995,3.742857,3.630537,Jumanji (1995),...,0,0,0,0,0,0,0,0,0,0
1,1,29,3.5,2005-04-02 23:31:16,3.950567,9.050289,1995,3.742857,3.630537,"City of Lost Children, The (Cité des enfants p...",...,0,0,0,0,1,0,1,0,0,0
2,1,32,3.5,2005-04-02 23:33:39,3.897763,10.713995,1995,3.742857,3.630537,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),...,0,0,0,0,1,0,1,1,0,0


In [3]:
print(f"Number of unique userId: {rating_enriched['userId'].nunique()}")
print(f"Number of unique movieId: {rating_enriched['movieId'].nunique()}")
print(f"Number of reviews: ~ {len(rating_enriched)/1000000:.1f} M")

Number of unique userId: 138383
Number of unique movieId: 12531
Number of reviews: ~ 19.9 M


### <u>0.2 Model evaluation </U>
In order to evaluate model performance and enable fair comparison with future approaches, we adopt a Leave-5-out strategy: for each user, the last 5 ratings are held out for testing, while the rest are used for training. This mirrors real-world scenarios where unseen items are recommended based on past behavior.
We assess accuracy using Root Mean Squared Error (RMSE) between predicted and actual ratings on the held-out items. This evaluation setup provides a consistent and realistic benchmark across all models.

The choice of holding out 5 ratings per user ensures a good balance between training size and evaluation coverage. Since we only include users with at least 20 ratings, this results in a minimum 75% train and 25% test split per user. Based on the user activity distribution observed in the eda.fe.ipynb notebook, this threshold offers sufficient signal for training while ensuring each user contributes to model evaluation.

Additionally with approximately 138,000 unique users in the final dataset, this strategy yields around 690,000 test points for evaluation, out of nearly 20 million total ratings, providing robust and reliable validation across the user base.

In [4]:
# Sort in term of review date and user id and then pick 10 most recent review for each userId
sorted_df = rating_enriched.sort_values(by=['userId','timestamp'], ascending=[True,True])
test_df = sorted_df.groupby('userId').tail(5)

# Build train df by removing the test_df rows from it
train_df = rating_enriched.drop(test_df.index).reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [5]:
# Assert correct length across users in the test set
assert test_df.groupby('userId').size().eq(5).all(), "Some users in test_df have ≠ 5 ratings"

In [6]:
# Manual Check of test_train split
user_id = 1

# All ratings for that user, sorted as in the previous setting
user_ratings = rating_enriched[rating_enriched['userId'] == user_id].sort_values('timestamp', ascending=True)

# Check that the 5 most recent ratings match test_df's entries for that user
expected_test_rows = user_ratings.tail(5).reset_index(drop=True)
actual_test_rows = test_df[test_df['userId'] == user_id].reset_index(drop=True)

# Assert that they match
assert expected_test_rows[['movieId', 'timestamp']].equals(actual_test_rows[['movieId', 'timestamp']])


## <u> 1. User-User Collaborative Filtering </u>

As previously explained, the main idea behind the **user–user collaborative filtering** approach is to recommend items to a target user by finding other users with similar tastes, then suggesting items they liked but the target user hasn't seen yet.

To compute similarities across users, we use Pearson **correlation** on the **user–item rating matrix**, which is structured as follows:
- Rows represent users (`userId`)
- Columns represent movies (`movieId`)
- Values are the ratings users assigned to each movie

This results in a very sparse matrix, as the dataset contains approximately 139,000 users and 12,000 movies, and most users rate only a small fraction of all available movies.

The prediction for how much a user $u$ will like an unseen item $i$ is computed using a **similarity-weighted average** of the ratings from the most similar users who have rated item $i$:

$$
\hat{r}_{u,i} = \bar{r}_u + \frac{\sum_{v \in N(u)} \text{sim}(u,v) \cdot (r_{v,i} - \bar{r}_v)}{\sum_{v \in N(u)} |\text{sim}(u,v)|}
$$

Where:
- $\hat{r}_{u,i}$ is the predicted rating for user $u$ on item $i$
- $\bar{r}_u$ is the average rating of user $u$
- $N(u)$ is the set of top-$K$ most similar users to $u$ who rated item $i$
- $\text{sim}(u,v)$ is the Pearson correlation between users $u$ and $v$
- $r_{v,i}$ is the rating that user $v$ gave to item $i$
- $\bar{r}_v$ is the average rating of user $v$

<br>

For the sake of efficiency and prediction stability, we set $K=20$, meaning that predictions are based on the 20 most similar users who have rated the item. This value offers a good trade-off: it is large enough to smooth out noise from individual ratings, while still focusing on the most relevant users.
Additionally, by setting the minimum number of reviews per movie to 25, we ensure that each item has enough rating data to support consistent top-K neighbor selection during prediction.

<br>

### <u>Preparation</u>

#### <u> 1.1.1 Build user-item rating matrix:</u> 

Given the large size of the user–item matrix, we use `csr_matrix` to store it efficiently in a sparse format.
This allows us to keep only the non-zero ratings and their positions, instead of allocating memory for the full (mostly empty) matrix. Since the sparse matrix stores only index-based positions, we keep the original userId and movieId mappings using LabelEncoder to translate between matrix indices and real IDs.

In [21]:
# Filter out unnecessary features
df_u_u = train_df[['userId','movieId','rating', 'title']].copy()

# Encode userId and movieId to ensure compatible indexing with csr_matrix
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
df_u_u['user_idx'] = user_encoder.fit_transform(df_u_u['userId'])
df_u_u['item_idx'] = movie_encoder.fit_transform(df_u_u['movieId'])

# Build spare matrix with efficient allocation of memory
user_item_sparse = csr_matrix(
    (df_u_u['rating'], (df_u_u['user_idx'], df_u_u['item_idx']))
)

# Compute transpose for easier acess during computation
item_user_sparse = user_item_sparse.T.tocsr()

#### <u> 1.1.2 Compute users rating mean:</u> 

In [22]:
# Compute mean of each users ($\hat{r}_u$)
user_means = df_u_u.groupby('userId')['rating'].mean()

#### <u> 1.1.3 Pearson computation function:</u> 

In [26]:
def compute_pearson(u_vec, v_vec):

    '''
    Compute pearson similarity across two users based on shared movies.
    '''

    # Find intersection of movies watched across user u and v
    common = np.intersect1d(u_vec.indices, v_vec.indices)

    # If not enough overalapping return 0
    if len(common) < 2:
        return 0
    
    # Compute similaries based on shared movies
    u_ratings = u_vec[:, common].toarray().flatten()
    v_ratings = v_vec[:, common].toarray().flatten()


     # Return 0 if either user has constant ratings
    if np.std(u_ratings) == 0 or np.std(v_ratings) == 0:
        return 0

    # Compute Pearson similarity
    sim, _ = pearsonr(u_ratings, v_ratings)
    return sim if not np.isnan(sim) else 0

#### <u>1.1.4 Prediction function on test set:</u>

In [27]:
def predict_rating(u_idx, i_idx, K=20):

    '''
    Predict the rating of user u_idx on item i_idx based on the K most similar users.
    '''
    
    # Get sparse rating vector of target user u
    u_vec = user_item_sparse[u_idx]

    # Get all users who rated item i
    users_who_rated_i = item_user_sparse[i_idx].indices

    similarities = []
    ratings = []

    for v_idx in users_who_rated_i:
        
        # Skip the target user
        if v_idx == u_idx:
            continue
        
        # Compute Pearson correlation between user u and neighbor v
        sim = compute_pearson(u_vec, user_item_sparse[v_idx])
        if sim <= 0:
            continue

        # Get user v's rating on item i
        r_vi = user_item_sparse[v_idx, i_idx]

        # Get user v's average rating
        r_v_mean = user_means.get(v_idx, 0)

        # Store similarity and the rating deviation
        similarities.append(sim)
        ratings.append((r_vi - r_v_mean, sim))

    # If no valid neighbors, fall back to user mean or 3.0
    if not ratings:
        return user_means.get(u_idx, 3.0)

    # Sort neighbors by absolute similarity and select top K
    ratings = sorted(ratings, key=lambda x: x[1], reverse=True)[:K]

    # Compute weighted average of rating deviations
    numerator = sum((r * s) for r, s in ratings)
    denominator = sum(abs(s) for _, s in ratings)

    # Avoid division by zero (in case similarities cancel out)
    if denominator == 0:
        return user_means.get(u_idx, 3.0)

    # Return user mean + similarity-weighted adjustment
    r_u_mean = user_means.get(u_idx, 3.0)
    return r_u_mean + numerator / denominator

### <u>1.2 Evaluation on test set </u>

In [30]:
# Encode test userId and movieId using the same encoders used on the train set
test_df['user_idx'] = user_encoder.transform(test_df['userId'])
test_df['item_idx'] = movie_encoder.transform(test_df['movieId'])

# Initialize prediction and ground truth lists
pred_ratings = []
true_ratings = []

# Loop through each row of the test set
for row in tqdm(test_df.itertuples(index=False), total=len(test_df)):
    u_idx = row.user_idx
    i_idx = row.item_idx
    true = row.rating

    # Predict rating
    pred = predict_rating(u_idx, i_idx, K=20)

    # Append results
    pred_ratings.append(pred)
    true_ratings.append(true)

# Compute RMSE
rmse = mean_squared_error(true_ratings, pred_ratings, squared= False)
print(f"Test RMSE: {rmse:.4f}")

  0%|          | 0/691915 [00:00<?, ?it/s]

KeyboardInterrupt: 